In [42]:
# import dependencies
import pandas as pd
# from geopy.geocoders import Nominatim
# import requests
# import json
# import gmaps
# from pandas import json_normalize
# from config import API_KEY

In [43]:
# Read in dataset
housing_data = pd.read_csv("Resources/Clean/San_Diego_Housing_Data.csv", index_col=0)
# Reset index to add back zipcodes
housing_data.reset_index(inplace=True)

# Make new df with San Diego zipcodes
zipcodesdf = housing_data[["Zip Code"]]
zipcodesdf

,Zip Code
0,91901
1,91902
2,91906
3,91910
4,91911
...,...
80,92131
81,92139
82,92154
83,92173


In [44]:
# Read in US zipcode database
us_zip = pd.read_csv("Resources/RAW/uszips.csv")

In [45]:
# Create dataframe for just zipcodes and lat/lon
lat_lng = us_zip[['zip', 'lat', 'lng']]
# Rename the zip column to zipcode so we can merge
lat_lng.rename(columns={'zip':'Zip Code'}, inplace=True)
lat_lng.head()

C:\Users\autum\AppData\Local\Temp\ipykernel_7800\3011836463.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lat_lng.rename(columns={'zip':'Zip Code'}, inplace=True)


,Zip Code,lat,lng
0,601,18.18027,-66.75266
1,602,18.36075,-67.17541
2,603,18.45744,-67.12225
3,606,18.16585,-66.93716
4,610,18.29110,-67.12243


In [47]:
# Merge dataframes on the zipcode
zip_lat = zipcodesdf.merge(lat_lng, on='Zip Code')
zip_lat.head(20)

,Zip Code,lat,lng
0,91901,32.81467,-116.71929
1,91902,32.67479,-117.00475
2,91906,32.66241,-116.47255
3,91910,32.63653,-117.06301
4,91911,32.60686,-117.04984
5,91913,32.62174,-116.98609
6,91914,32.66604,-116.95426
7,91915,32.62266,-116.95013
8,91932,32.56923,-117.11798
9,91935,32.70214,-116.78721


In [48]:
# Bring the index backto zipcode so we can iterate through safely.
zip_lat.set_index(['Zip Code'], inplace=True)

In [19]:
# Setting our API parameters
params = {
    "radius": 5000,
    "type": 'supermarket',
    "key": API_KEY
}

In [36]:
# Iterate the responses and filter out market names in a zipcode.
market_list = []

for index, row in zip_lat.iterrows():
    lat = zip_lat.loc[index, 'lat']
    lng = zip_lat.loc[index, 'lng']
    
    
    params["location"] = f"{lat},{lng}"
    
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    markets = requests.get(base_url,params=params).json()

    market_list.extend(markets.get('results'))


In [24]:
# Confirm we got results
len(market_list)

517

In [56]:
# Confirm we got coordinates for each market in a zipcode
market_list[0]['geometry']['location']

{'lat': 32.6389997, 'lng': -117.050924}

In [41]:
# Pull business operation status from our JSON
df2 = pd.DataFrame([x['business_status'] for x in market_list])
df2.set_axis(['business_status'], axis='columns', inplace=True)
df2

,0
0,OPERATIONAL
1,OPERATIONAL
2,OPERATIONAL
3,OPERATIONAL
4,OPERATIONAL
...,...
512,OPERATIONAL
513,OPERATIONAL
514,CLOSED_TEMPORARILY
515,OPERATIONAL


In [66]:
# Pull market names from our JSON
df3 = pd.DataFrame([x['name'] for x in market_list])
df3.set_axis(['name'],axis='columns', inplace=True)
df3

,name
0,Smart & Final Extra!
1,Sprouts Farmers Market
2,Carnival Supermarket
3,Target Grocery
4,99 Ranch Market
...,...
512,Abarrotes La Tepexpense
513,Waldo's
514,Farmamil chaparral
515,Abarrotes La Placita


In [67]:
# Pull lat/lon from our JSON
df4 = pd.DataFrame([x['geometry']['location'] for x in market_list])
df4

,lat,lng
0,32.639000,-117.050924
1,32.627807,-117.074994
2,32.621859,-117.072892
3,32.653616,-117.065593
4,32.629342,-117.040931
...,...,...
512,32.538416,-117.066559
513,32.534882,-117.040981
514,32.540419,-117.031336
515,32.535808,-117.037852


In [68]:
# Merge all our dataframes into one
final_market_df = pd.concat([df2, df3, df4], axis = 1)
final_market_df

,business_status,name,lat,lng
0,OPERATIONAL,Smart & Final Extra!,32.639000,-117.050924
1,OPERATIONAL,Sprouts Farmers Market,32.627807,-117.074994
2,OPERATIONAL,Carnival Supermarket,32.621859,-117.072892
3,OPERATIONAL,Target Grocery,32.653616,-117.065593
4,OPERATIONAL,99 Ranch Market,32.629342,-117.040931
...,...,...,...,...
512,OPERATIONAL,Abarrotes La Tepexpense,32.538416,-117.066559
513,OPERATIONAL,Waldo's,32.534882,-117.040981
514,CLOSED_TEMPORARILY,Farmamil chaparral,32.540419,-117.031336
515,OPERATIONAL,Abarrotes La Placita,32.535808,-117.037852


In [ ]:
# Drop rows that say closed
operationalmarkets = final_market_df.loc(final_market_df['business_status'] == "OPERATIONAL")

In [69]:
# exporting file
from pathlib import Path

filepath = Path("Resources/Clean/San_Diego_Markets.csv")
operationalmarkets.to_csv(filepath)